# 0. Setup

Imports core libraries for graph modeling, visualization, data handling, and API interaction.

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import sys
import json
import os

from datetime import datetime
from itertools import combinations
from openai import OpenAI
from collections import defaultdict
from pprint import pprint
from itertools import combinations
from google.colab import drive

Mount google drive to store experiments persistently:

In [ ]:
drive.mount('/content/drive')

path = 'MSc AI Polimi/Multidisciplinary Project/Implementation/Experiments'

os.chdir(f'/content/drive/MyDrive/{path}')
os.getcwd()

**API Configuration and Client Setup**  
Defines model, endpoint, and client configurations for managing OpenAI API access.

**Attributes:**

- `models`: Maps model names to their identifiers.
- `base_urls`: Stores base API endpoints.
- `clients_details`: A list of dictionaries, each containing:
  - `base_url`: The endpoint for API access.
  - `api_key`: A unique API key used for authenticating requests.

**clients**:  
A list of initialized `OpenAI` client objects, each created using the respective API key and base URL from `clients_details`.  
Useful for distributing requests across multiple keys to avoid rate limits.

In [ ]:
models = {
    'DeepSeek-V3-0324': 'deepseek/deepseek-chat-v3-0324:free'
}

base_urls = {
    'OpenRouter': 'https://openrouter.ai/api/v1',
}

clients_details = [
    {
        'base_url': base_urls['OpenRouter'],
        'api_key': ""
    },
    {
        'base_url': base_urls['OpenRouter'],
        'api_key': ""
    },

]

clients = [
    OpenAI(
        base_url=client['base_url'],
        api_key=client['api_key']
    ) for client in clients_details
]

**Client Rotation Logic**  
Handles active API client selection and rotation to avoid request limit issues.

**Attributes:**

- `current_client_idx`: Index of the currently active client.
- `current_client`: The active `OpenAI` client object used for API calls.

**Functions:**

- `update_current_client()`:  
  Increments the `current_client_idx` to point to the next client in the list (with wrap-around).  
  Updates `current_client` accordingly to maintain uninterrupted API access.

In [ ]:
current_client_idx = 0
current_client = clients[current_client_idx]

def update_current_client():
  global current_client_idx, current_client
  current_client_idx = (current_client_idx + 1) % len(clients)
  current_client = clients[current_client_idx]

# 1. Social Network Creation

**Agent** Class  
Represents an individual entity with a defined persona (e.g., name, traits, background).

**Attributes:**

- `persona`: A dictionary containing descriptive attributes of the agent (e.g., `{"Name": "Alice", "Role": "Teacher"}`).

In [ ]:
class Agent:
  def __init__(self, persona: dict):
    self.persona = persona

**SocialNetwork** Class  
Represents a network of agents using a graph structure. Enables group formation, visualization, and interaction modeling.

**Attributes:**

- `agents`: List of `Agent` instances.
- `network`: Undirected graph (using NetworkX) with agents as nodes.
- `groups`: List of created groups.

**Methods:**

- `__create_network()`: Initializes a graph with nodes for each agent.
- `draw_network()`: Displays the network graph with labels.
- `create_group(members, model)`:
  - Connects selected agents.
  - Creates a `Group` with those agents and the specified model.
  - Adds it to the `groups` list.

In [ ]:
class SocialNetwork:
  def __init__(self, agents: list[Agent]):
    self.agents = agents
    self.network = self.__create_network()
    self.groups = []

  def __create_network(self):
    network = nx.Graph()
    network.add_nodes_from(range(len(self.agents)))
    return network

  def draw_network(self):
    nx.draw(self.network, with_labels=True)
    plt.show()

  def create_group(self, members: list[int], model: str):
    connections = list(combinations(members, 2))
    self.network.add_edges_from(connections)

    members = [self.agents[member] for member in members]
    group = Group(members, model)
    self.groups.append(group)

    return group

**Group** Class  
Manages a group of agents, facilitates chat-based interactions, tracks message history, and supports AI-generated communication.

**Attributes:**

- `members`: List of `Agent` instances in the group.
- `model`: The language model used for generating responses.
- `chats`: List of chat records, each containing a sender, recipients, and a message.

**Methods:**

- `add_chat(member, message)`:  
  Adds a chat entry from the given agent to all other group members.

- `get_members_details(indices)`:  
  Returns the `persona` dictionaries of specified members by their indices.

- `get_list_messages(member)`:  
  Returns a list of messages sent by the specified agent.

- `get_chats_history(last_n)`:  
  Returns the most recent `n` chat entries in a structured format (with sender, recipients, and message).

- `send_message(sender_idx, prompt_start, len_chat_memory)`:  
  Constructs a conversation prompt using agent personas and chat history, sends it to the model, and retrieves a generated message. Rotates clients if a request fails.

- `communicate(prompt_start, rounds, len_chat_memory)`:  
  Runs multi-round interaction where each agent sends a message per round. New messages are added to the chat log and printed after each exchange.

In [ ]:
class Group:
  def __init__(self, members: list[Agent], model: str):
    self.members = members
    self.model = model
    self.chats = [] # [{'sender': <Agent>, 'recipients': [<Agent>, ...], 'message': <str>}, ...]

  def add_chat(self, member: Agent, message: str):
    recipients = [m for m in self.members if m != member]
    chat = {'sender': member, 'recipients': recipients, 'message': message}
    self.chats.append(chat)

  def get_members_details(self, indices: list[int]):
    return [self.members[i].persona for i in indices]

  def get_list_messages(self, member: Agent):
    return [chat['message'] for chat in self.chats if chat['sender'] == member]

  def get_chats_history(self, last_n: int|None = None):
    num_all_chats = len(self.chats)
    last_n = num_all_chats if (last_n is None) else min(last_n, num_all_chats)

    chats_history = [
        {
            'sender': chat['sender'].persona['Name'],
            'recipients': [recipient.persona['Name'] for recipient in chat['recipients']],
            'message': chat['message']
        } for chat in self.chats[-last_n:]
    ]

    return chats_history

  def send_message(self, sender_idx: int, prompt_start: str, len_chat_memory: int|None = None):
    sender_details = self.get_members_details([sender_idx])

    receivers_indices = [j for j in range(len(self.members)) if j != sender_idx]
    receivers_details = self.get_members_details(receivers_indices)

    prompt = f"You are {sender_details}, engaging in a friendly conversation with: {receivers_details}.\n"
    prompt += f"This is the chat history so far: {self.get_chats_history(len_chat_memory)}.\n"
    prompt += """Generate only the next message in the conversation, using your natural personality and communication style. \
    Assume this is an online chat on social media, not an in-person conversation. \
    If the chat history is empty, begin the conversation yourself—do not simulate the other participant’s message.\n"""
    prompt += prompt_start

    def get_client_response():
      try:
        return current_client.chat.completions.create(
            model=self.model,
            messages=[
                {
                    'role': 'user',
                    'content': prompt
                }
            ],
        )
      except:
        return None

    num_failed_tries = 0
    while True:
        response = get_client_response()
        if response is not None:
            break
        num_failed_tries += 1
        if num_failed_tries > len(clients):
            sys.exit('No response from any client')
        update_current_client()

    return response.choices[0].message.content.strip()

  def communicate(self, prompt_start: str, rounds: int, len_chat_memory: int|None = None):
    for _ in range(rounds):
      for sender_idx in range(len(self.members)):
        message = self.send_message(sender_idx, prompt_start, len_chat_memory)
        self.add_chat(self.members[sender_idx], message)
        pprint(self.get_chats_history(1))

# 2. Experiments

**Agent Initialization and Social Network Setup**  
Defines a set of agent personas and constructs a social network based on them.

**Attributes:**

- `persona_keys`: List of attributes used to define each agent.
- `personas`: A list of individual personas, each represented as a list of attribute values.
- `agents`: A list of `Agent` objects created by mapping `persona_keys` to each persona's values.
- `social_network`: An instance of the `SocialNetwork` class initialized with the generated agents.


In [ ]:
persona_keys = ['Name', 'Gender', 'Age', 'Economic Status', 'Occupation']

personas = [
  ['Kayla', 'Female', 'Teen', 'Working Class', 'TikTok Influencer'],
  ['Morgan', 'Nonbinary', 'Adult', 'Upper-Middle', 'Corporate Lawyer'],
  ['Frank', 'Male', 'Elderly', 'Poor', 'Uber Driver'],
  ['Karen', 'Female', 'Middle-Aged', 'Middle Class', 'Politician (Controversial)'],
  ['Leo', 'Male', 'Young Adult', 'Lower Class', 'Activist (Environmental)']
]

agents = [Agent(dict(zip(persona_keys, persona))) for persona in personas]

social_network = SocialNetwork(agents)

**Experiment Configuration**  
Specifies the model, group combinations, prompts, and communication settings for simulation.

**Attributes:**

- `model` (static): The selected language model used for agent communication.
- `groups_length` (static): Number of agents per group (used to form combinations).
- `groups` (dynamic): All possible combinations of agent indices of length `groups_length`.
- `prompts_start` (dynamic): List of initial prompts for each experimental scenario.
- `communication_rounds` (static): Number of communication rounds each group will go through.
- `communication_memory` (dynamic): List of memory settings; `None` allows access to full chat history, while `10` limits it to the most recent 10 messages.

In [ ]:
model = models['DeepSeek-V3-0324']

groups_length = 4
groups = list(map(list, combinations(range(len(personas)), groups_length)))

prompts_start = [
    {
        'title': "work_life_balance",
        'prompt': "The topic is work–life balance. Share how you handle stress from your job, and whether you believe in strict boundaries or letting work and personal life blend naturally."
    },
    {
        'title': "failure_reflection",
        'prompt': "Describe a time you failed or faced a major setback. How did you cope with it emotionally or logically, and what did you take away from the experience?"
    },
    {
        'title': "content_type",
        'prompt': "What kind of content grabs your attention the most — real-world stories, fantasy, how-to guides, philosophical debates, or emotional journeys? Why?"
    },
]

communication_rounds = 20
communication_memory = [None, 10] # `None` means having access to all previous chats

**Experiment Execution Loop**  
Runs communication experiments for each group, prompt, and memory configuration, and stores the resulting chat history.

**Process:**

- Iterates over all combinations of:
  - `group`: A subset of agent indices.
  - `prompt_start`: A dictionary containing a `title` and `prompt` for initializing the conversation.
  - `len_chat_memory`: The number of previous messages to include as context (can be limited or `None` for full history).

**Workflow per Experiment:**

1. **Naming**:  
   Generates a unique experiment name using the group members, prompt title, and memory length.

2. **Group Creation**:  
   Uses `social_network.create_group()` to instantiate a group of agents for the current configuration.

3. **Network Visualization**:  
   Calls `social_network.draw_network()` to display the current agent connections.

4. **Communication Simulation**:  
   Starts the conversation using `group_obj.communicate()` with the given prompt and memory setting.

5. **Result Saving**:  
   Stores the full chat history to a `.json` file named after the current experiment.

**Note**:  
This setup allows scalable experimentation across various social setups and memory constraints.

In [ ]:
for group in groups:
  for prompt_start in prompts_start:
    for len_chat_memory in communication_memory:
      experiment_name = f"{group}-{prompt_start['title']}-{len_chat_memory}"

      group_obj = social_network.create_group(group, model)

      social_network.draw_network()

      group_obj.communicate(prompt_start['prompt'], communication_rounds, len_chat_memory)

      with open(f"{experiment_name}.json", 'w') as f:
        json.dump(group_obj.get_chats_history(), f)